In [331]:
# Block 1
# RAG with title - total article search

In [332]:
api_key = 'up_JQDg7exNaCa9WsUszuo1v3dmWc2LZ'

In [333]:
from langchain_upstage import UpstageEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

llm = ChatUpstage(api_key=api_key, model='solar-1-mini-chat', temperature=0) # added temperature for random seed

embeddings = UpstageEmbeddings(
    api_key=api_key,
    model="embedding-query"
)

In [334]:
query = "사회체육학과가 취득하는 학위 명은?"

| Method1: Cosine similarity with question - title

In [335]:
def search_title_similarity(embedded_title, query):
    
    query_vec = embeddings.embed_query(query)
    
    with open(embedded_title, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
        
    similarities = []
    
    for idx, entry in data.items():
        title_vec = entry.get("title")
        
        query_vec = np.array(query_vec)
        title_vec = np.array(title_vec[0])
        
        # print(query_vec.shape, title_vec.shape)
        
        similarity = cosine_similarity([query_vec], [title_vec])[0][0]
        similarities.append((idx, similarity))
        
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_3_indices = [sim[0] for sim in similarities[:3]]
    
    return top_3_indices

In [336]:
embedded_title = "../docs/embedded_title.json"
embedded_doc = "../docs/embedded_doc1.json"

top3 = search_title_similarity(embedded_title, query)
print(top3[0], top3[1], top3[2])

17 2 3


In [337]:
def get_context_text(index, doc_1):
    with open(doc_1, 'r', encoding = 'utf-8') as infile:
        data = json.load(infile)
        
    # print(data["0"].get("content"))
    
    return data[str(index)].get("content")

In [338]:
get_context_text(0, "../docs/doc_1.json")

'- 제정 날짜: 1946.8.15\n- 개정 날짜: 2017.8.16'

In [339]:
prompt_template = PromptTemplate.from_template(
    """
    Please provide the most correct answer BASED ON THE CONTEXT.
    
    Response format:
    <Start>
    [Answer]: (A) answer
    [Reason]: Your short reason why
    <End>
    
    Now, here are the question and context:
    Question: {question}
    Context: {context}
    
    If the answer is NOT found, you MUST respond with "The information is not present in the context."
    """
)

In [340]:
chain = prompt_template | llm

context = get_context_text(top3[0], "../docs/doc_1.json")
answer = chain.invoke({"question": query, "context": context})

In [341]:
print(context)
print(answer.content)

- 부칙(2006. 6. 28  개정)
 ① (시행일)  이 학칙은 2007년 3월 1일부터 시행한다. 다만, 제8조 제4항 제18호는 2005년 8월 13일부터, 제50조 [별표 2] 중 바이오인포매틱스 연계전공은 2006년 9월 1일부터 시행한다. 
② (경과조치)  제50조 [별표 2]와 관련하여, 편제 조정으로 인하여 이 학칙 시행당시 소속대학이 변경되거나 명칭이 변경된 학과(전공) 또는 신입생 모집이 중단된 학과(전공)의 재적생에 대하여는 구 학칙에 따라 다음과 같이 학사학위를 수여한다.  
[
    {'대학': '사회과학대학', '학과 또는 전공': '사회복지학', '학위의 종류': '문학사'},
    {'대학': '자연과학대학', '학과 또는 전공': '분자생명과학', '학위의 종류': '이학사'},
    {'대학': '공과대학', '학과 또는 전공': '컴퓨터학', '학위의 종류': '공학사'},
    {'대학': '공과대학', '학과 또는 전공': '정보통신학', '학위의 종류': '공학사'},
    {'대학': '공과대학', '학과 또는 전공': '건축학(4년제)', '학위의 종류': '공학사'},
    {'대학': '공과대학', '학과 또는 전공': '건축학(5년제)', '학위의 종류': '건축학사'},
    {'대학': '공과대학', '학과 또는 전공': '환경학', '학위의 종류': '공학사'},
    {'대학': '음악대학', '학과 또는 전공': '건반악기', '학위의 종류': '음악학사'},
    {'대학': '음악대학', '학과 또는 전공': '관현악', '학위의 종류': '음악학사'},
    {'대학': '음악대학', '학과 또는 전공': '성악', '학위의 종류': '음악학사'},
    {'대학': '음악대학', '학과 또는 전공': '교회음악', '학위의 종류': '음악학사'},
    {'대학': '음악대학', '학과 또는 전공': '작곡', '학위의 종류': '음악학사'},
    {'대학': '음악대

In [342]:
prompt_template = PromptTemplate.from_template(
    """
    Is there a contradiction in this context? If so, why?
    Context: {context}
    """
)

In [343]:
chain = prompt_template | llm

response = chain.invoke({"question": query, "context": answer.content})

In [344]:
print(response.content)

아니요, 이 문맥에서는 모순이 없습니다. 문맥에서는 체육과학대학의 사회체육학 전공이 이학사 학위를 취득한다고 명확히 언급하고 있습니다. 이학사는 과학 분야의 학사 학위를 의미하며, 사회체육학 전공이 과학 분야에 속하기 때문에 이학사 학위를 취득하는 것은 일관성이 있습니다.
